<a href="https://colab.research.google.com/github/MuhammadFadhilArkan/Techtest/blob/main/Q2/Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Necessary Library

In [1]:
!pip install PyPDF2
!pip install tika
!pip install tabula-py

     |████████████████████████████████| 77 kB 2.0 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=1de147196dfe0f92055edffd5dede1d051156438eb7d716db246d198d5a47a32
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32891 sha256=eccfe8a0c70533052a40386e358672f1ccdbc140a6d417ced4c8165b2697f457
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika
     |████████████████████████████████| 12.0 MB 106 kB/s 


Import the Library

In [2]:
import PyPDF2 
import tabula
import tika
from tika import parser
import numpy as np

Define filepath of the pdf file

In [3]:
filepath = "/content/drive/MyDrive/CAD IT/Machine Learning Application Engineer Test 2021 V1.1/Q2.pdf"

Determine the total number of pages in the pdf file

In [4]:
# creating a pdf file object 
pdfFileObj = open(filepath, 'rb') 
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
# get number of pages in pdf file 
num_pages = pdfReader.numPages

Extract table data from the pdf file

In [5]:
dfs = []
for i in range (1,num_pages+1):
  df = tabula.read_pdf(
    filepath,
    pages=i,
    lattice=True,
    silent=True
    )
  dfs.append(df)

Scrap information about Description and Possible Root Cause from the extracted table

In [6]:
descs = []
prcs = []
for i in range (num_pages):
  dum = dfs[i][0]
  dum = dum.dropna(axis = 0, how = 'all')
  dum = dum.dropna(axis = 1, how = 'all')
  dum.reset_index(drop=True, inplace=True)
  dum = dum.T.reset_index(drop=True).T
  is_contain = dum[0].str.contains('Description')
  if not np.any(is_contain) :
    dum = dum.drop(0, axis=1) 
    dum = dum.T.reset_index(drop=True).T
  desc = dum[dum[0]=="Description"]
  desc = desc.iloc[0,1]
  prc = dum[dum[0]=="Possible Root\rcause"]
  prc = prc.iloc[0,1]
  descs.append(desc)
  prcs.append(prc)

Extract the page number of each page in the pdf file

In [ ]:
PDF_Parse = parser.from_file(filepath)
content = PDF_Parse ['content']

clue = content.split("Service Diagnosis")
clue = np.array(clue)

page_numbers = []
for i in range (1,len(clue)):
  page_number = clue[i][3:5]
  page_numbers.append(page_number)

Preprocess all the data before importing to a csv file

In [8]:
prcs = np.array(prcs)
descs = np.array(descs)
page_numbers = np.array(page_numbers)

page_numbers = page_numbers.reshape(-1,1)
descs = descs.reshape(-1,1)
prcs = prcs.reshape(-1,1)

content = np.concatenate((page_numbers, descs,prcs), axis=1)

header = np.array([["Page numbers", "Description", "Possible root cause"]])

Q2 = np.concatenate((header,content),axis=0)

Create the csv file

In [9]:
import csv
with open('/content/drive/MyDrive/CAD IT/Q2/Q2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(Q2)

For sanity check, Read the csv file that we've been created

In [10]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/CAD IT/Q2/Q2.csv")
display(df)

,Page numbers,Description,Possible root cause
0,35,INDOOR PCB ABNORMALITY,1.Faulty indoor PCB.\r2.Faulty connector conne...
1,36,ANTIFREEZE PROTECTION OR HIGH PRESSURE CONTROL,"1.Indoor air filter, heat exchanger block due ..."
2,37,INDOOR FAN MOTOR ABNORMALITY,"1.Indoor fan motor winding short, or the motor..."
3,38,INDOOR ROOM THERMISTOR ABNORMALITY,"1.Thermistor, connector faulty.\r2.Indoor PCB ..."
4,39,OUTDOOR PCB ABNORMALITY,1.Micro Controller program run-away due to ext...
5,40,COMPRESSOR OVERLOAD,1.Refrigerant Shortage.\r2.4 way valve malfunc...
6,41,COMPRESSOR OVERLOAD,1.Refrigerant Shortage.\r2.4 way valve malfunc...
7,42,COMPRESSOR LOCK/START-UP ABNORMALITY,1.Compressor locked.\r2.Compressor harness dis...
8,43,OUTDOOR FAN MOTOR LOCK,1.Fan motor breakdown.\r2.Harness or connector...
9,44,AC INPUT OVER CURRENT,1.Over current due to compressor failure.\r2.O...
